In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


import sqlite3
import random
import os

random.seed(137)
np.random.seed(137)

pd.options.mode.chained_assignment = None  # default="warn"
%matplotlib inline

In [2]:
def is_numeric(x):
    try:
        float(x)
        return True
    except:
        return False

## Read data

In [3]:
data_dir = "/N/project/waveform_mortality/shared/datasets/Clinical_datasets/eicu-2.0"

In [4]:
nurse_charting = pd.read_csv(os.path.join(data_dir, "nurseCharting.csv"))
nurse_charting

,nursingchartid,patientunitstayid,nursingchartoffset,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartcelltypevalname,nursingchartvalue
0,221469923,141924,5714,5714,Vital Signs,Temperature,Temperature (C),36.6
1,277425167,141924,12964,12964,Other Vital Signs and Infusions,Genitourinary Assessment,Value,X
2,236464484,141924,3844,3844,Vital Signs,Heart Rate,Heart Rate,97
3,253552049,141924,124,124,Vital Signs,Non-Invasive BP,Non-Invasive BP Systolic,106
4,147240923,141924,1489,1489,Other Vital Signs and Infusions,MAP (mmHg),Value,71
...,...,...,...,...,...,...,...,...
151604227,80599950,141924,4774,4774,Other Vital Signs and Infusions,Patient s Comfort/Function (Pain) GOAL At Rest,Value,3
151604228,180586115,141924,5714,5714,Vital Signs,Non-Invasive BP,Non-Invasive BP Systolic,100
151604229,95065377,141924,5269,5269,Vital Signs,Temperature,Temperature Location,ORAL
151604230,168147793,141924,9064,9064,Vital Signs,Temperature,Temperature Location,ORAL


In [5]:
# mask = nurse_charting["nursingchartvalue"].apply(is_numeric)
# nurse_charting = nurse_charting[mask]
# nurse_charting["nursingchartvalue"] = nurse_charting["nursingchartvalue"].astype(float)
# nurse_charting
nurse_charting["nursingchartvalue"] = nurse_charting["nursingchartvalue"].str.lower()

In [6]:
resp_chart = pd.read_csv(os.path.join(data_dir, "respiratoryCharting.csv"))
resp_chart

/tmp/ipykernel_3091440/1112102243.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  resp_chart = pd.read_csv(os.path.join(data_dir, "respiratoryCharting.csv"))


,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
0,3,221468,4679,4679,respFlowCareData,RT Vent On/Off,Continued
1,4,221468,4679,4679,respFlowSettings,FiO2,40
2,5,221468,4679,4679,respFlowSettings,Vent Rate,14
3,6,221468,4679,4679,respFlowSettings,Tidal Volume (set),450
4,7,221468,4679,4679,respFlowSettings,TV/kg IBW,5.4720
...,...,...,...,...,...,...,...
20168171,255567518,3335097,15149,15149,respFlowPtVentData,Peak Insp. Pressure,14
20168172,255567519,3335097,15149,15149,respFlowPtVentData,Exhaled MV,7.9
20168173,255567520,3335097,15149,15149,respFlowPtVentData,Mean Airway Pressure,6
20168174,255567527,3331083,126,126,respFlowSettings,FiO2,40


In [7]:
resp_chart["respchartvaluelabel"] = resp_chart["respchartvaluelabel"].str.lower()

In [8]:
treatment = pd.read_csv(os.path.join(data_dir, "treatment.csv"))
treatment

,treatmentid,patientunitstayid,treatmentoffset,treatmentstring,activeupondischarge
0,8399138,242040,198,cardiovascular|hypertension|angiotensin II rec...,False
1,8626134,242040,198,cardiovascular|myocardial ischemia / infarctio...,False
2,8517569,242040,198,infectious diseases|medications|therapeutic an...,False
3,9597686,242040,616,cardiovascular|non-operative procedures|diagno...,False
4,9334096,242040,618,infectious diseases|medications|therapeutic an...,True
...,...,...,...,...,...
3688740,82519717,3353251,291,pulmonary|radiologic procedures / bronchoscopy...,False
3688741,82608952,3353251,291,pulmonary|ventilation and oxygenation|mechanic...,False
3688742,82541591,3353254,41,cardiovascular|intravenous fluid|normal saline...,True
3688743,82230708,3353254,41,cardiovascular|intravenous fluid|blood product...,True


In [9]:
treatment["treatmentstring"] = treatment["treatmentstring"].str.lower()

In [10]:
icu_stays = pd.read_csv(os.path.join(data_dir, "patient.csv"))
icu_stays

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141178,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870
2,141179,128927,Female,52,Caucasian,60,83,NaN,162.6,08:56:00,...,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,...,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276
4,141196,128943,Male,71,Caucasian,67,109,NaN,162.6,20:21:00,...,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200854,3353235,2743084,Male,50,Caucasian,458,1109,"CHF, congestive heart failure",175.3,04:55:00,...,Emergency Department,1,admit,90.0,99.2,23:18:00,1069,Telemetry,Alive,035-16382
200855,3353237,2743086,Female,79,Caucasian,458,1106,"Embolus, pulmonary",162.6,01:45:00,...,Direct Admit,1,admit,78.4,81.4,23:08:00,1269,Step-Down Unit (SDU),Alive,035-751
200856,3353251,2743099,Male,73,African American,458,1104,Cardiac arrest (with or without respiratory ar...,177.8,12:51:00,...,Emergency Department,1,admit,102.0,96.2,23:16:00,16259,Telemetry,Alive,035-5166
200857,3353254,2743102,Male,81,Caucasian,459,1108,"Bleeding, lower GI",185.4,07:43:00,...,Emergency Department,1,admit,83.9,92.9,19:25:00,431,Step-Down Unit (SDU),Alive,035-19511


## Ventilation settings 0

### assemble ventilation from nurse

In [11]:
vent_settings_nurse = nurse_charting[["patientunitstayid", "nursingchartoffset", "nursingchartvalue"]]
vent_settings_nurse = vent_settings_nurse.rename(columns={"nursingchartoffset": "charttime", "nursingchartvalue": "string"})
vent_settings_nurse["activeupondischarge"] = None
vent_settings_nurse

,patientunitstayid,charttime,string,activeupondischarge
0,141924,5714,36.6,None
1,141924,12964,x,None
2,141924,3844,97,None
3,141924,124,106,None
4,141924,1489,71,None
...,...,...,...,...
151604227,141924,4774,3,None
151604228,141924,5714,100,None
151604229,141924,5269,oral,None
151604230,141924,9064,oral,None


### from resp

In [12]:
vent_settings_resp = resp_chart[["patientunitstayid", "respchartentryoffset", "respchartvaluelabel"]]
vent_settings_resp = vent_settings_resp.rename(columns={"respchartentryoffset": "charttime", "respchartvaluelabel": "string"})
vent_settings_resp["activeupondischarge"] = None
vent_settings_resp

,patientunitstayid,charttime,string,activeupondischarge
0,221468,4679,rt vent on/off,None
1,221468,4679,fio2,None
2,221468,4679,vent rate,None
3,221468,4679,tidal volume (set),None
4,221468,4679,tv/kg ibw,None
...,...,...,...,...
20168171,3335097,15149,peak insp. pressure,None
20168172,3335097,15149,exhaled mv,None
20168173,3335097,15149,mean airway pressure,None
20168174,3331083,126,fio2,None


In [13]:
mask = resp_chart["respchartvaluelabel"].isin(["o2 device", "respiratory device", "ventilator type", "oxygen delivery method"])
vent_settings_resp_oxygen_device = resp_chart.loc[mask, ["patientunitstayid", "respchartentryoffset", "respchartvalue"]]
vent_settings_resp_oxygen_device = vent_settings_resp_oxygen_device.rename(columns={"respchartentryoffset": "charttime", "respchartvalue": "string"})
vent_settings_resp_oxygen_device["activeupondischarge"] = None
vent_settings_resp

,patientunitstayid,charttime,string,activeupondischarge
0,221468,4679,rt vent on/off,None
1,221468,4679,fio2,None
2,221468,4679,vent rate,None
3,221468,4679,tidal volume (set),None
4,221468,4679,tv/kg ibw,None
...,...,...,...,...
20168171,3335097,15149,peak insp. pressure,None
20168172,3335097,15149,exhaled mv,None
20168173,3335097,15149,mean airway pressure,None
20168174,3331083,126,fio2,None


### from treatment

In [14]:
vent_settings_treatment = treatment[["patientunitstayid", "treatmentoffset", "treatmentstring", "activeupondischarge"]]
vent_settings_treatment = vent_settings_treatment.rename(columns={"treatmentoffset": "charttime", "treatmentstring": "string"})
vent_settings_treatment

,patientunitstayid,charttime,string,activeupondischarge
0,242040,198,cardiovascular|hypertension|angiotensin ii rec...,False
1,242040,198,cardiovascular|myocardial ischemia / infarctio...,False
2,242040,198,infectious diseases|medications|therapeutic an...,False
3,242040,616,cardiovascular|non-operative procedures|diagno...,False
4,242040,618,infectious diseases|medications|therapeutic an...,True
...,...,...,...,...
3688740,3353251,291,pulmonary|radiologic procedures / bronchoscopy...,False
3688741,3353251,291,pulmonary|ventilation and oxygenation|mechanic...,False
3688742,3353254,41,cardiovascular|intravenous fluid|normal saline...,True
3688743,3353254,41,cardiovascular|intravenous fluid|blood product...,True


### ventilation assembled for assigning oxygen therapy type

In [15]:
vent_settings = pd.concat([vent_settings_nurse, vent_settings_resp, vent_settings_resp_oxygen_device, vent_settings_treatment], ignore_index=True)
vent_settings

/N/slate/xl95/miniconda3/envs/gnn/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
/N/slate/xl95/miniconda3/envs/gnn/lib/python3.10/site-packages/IPython/core/formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,patientunitstayid,charttime,string,activeupondischarge
0,141924,5714,36.6,None
1,141924,12964,x,None
2,141924,3844,97,None
3,141924,124,106,None
4,141924,1489,71,None
...,...,...,...,...
175999312,3353251,291,pulmonary|radiologic procedures / bronchoscopy...,False
175999313,3353251,291,pulmonary|ventilation and oxygenation|mechanic...,False
175999314,3353254,41,cardiovascular|intravenous fluid|normal saline...,True
175999315,3353254,41,cardiovascular|intravenous fluid|blood product...,True


In [16]:
vent_settings_interest = vent_settings.copy(True)

In [17]:
vent_settings_interest = vent_settings_interest[~vent_settings_interest["string"].isna()]

### invasive ventilation

In [18]:
type_4_ventilation_categories_1 = ["plateau pressure", "postion at lip", "position at lip", "pressure control"]
type_4_ventilation_1 = vent_settings_interest[vent_settings_interest["string"].isin(type_4_ventilation_categories_1)]
type_4_ventilation_1

,patientunitstayid,charttime,string,activeupondischarge
151604239,221468,4679,plateau pressure,None
151604286,181209,34983,plateau pressure,None
151604298,189335,13086,plateau pressure,None
151604304,189335,13086,pressure control,None
151604330,172049,916,plateau pressure,None
...,...,...,...,...
171771806,3335372,7367,pressure control,None
171771924,3328229,13385,pressure control,None
171772105,3342840,9559,pressure control,None
171772127,3338334,14205,pressure control,None


In [19]:
type_4_ventilation_categories_2 = [
    "set vt", "sputum", "rsbi", "tube", "ett", "endotracheal", "tracheal suctioning", "tracheostomy",
    "tracheostomy", "reintubation", "assist controlled", "volume controlled", "pressure controlled", "trach collar"
]
type_4_ventilation_2 = vent_settings_interest[vent_settings_interest["string"].str.contains(("|").join(type_4_ventilation_categories_2))]
type_4_ventilation_2

,patientunitstayid,charttime,string,activeupondischarge
18155515,248267,22,trach collar,None
18176114,249125,19571,trach collar,None
18176116,249125,18071,trach collar,None
18176588,249125,18131,trach collar,None
18176671,249125,16931,trach collar,None
...,...,...,...,...
175999242,3353216,1376,pulmonary|radiologic procedures / bronchoscopy...,True
175999294,3353251,12374,pulmonary|radiologic procedures / bronchoscopy...,True
175999306,3353251,186,pulmonary|radiologic procedures / bronchoscopy...,False
175999308,3353251,108,pulmonary|radiologic procedures / bronchoscopy...,False


In [20]:
type_4_ventilation = pd.concat([type_4_ventilation_1, type_4_ventilation_2], ignore_index=True)
type_4_ventilation

,patientunitstayid,charttime,string,activeupondischarge
0,221468,4679,plateau pressure,None
1,181209,34983,plateau pressure,None
2,189335,13086,plateau pressure,None
3,189335,13086,pressure control,None
4,172049,916,plateau pressure,None
...,...,...,...,...
1072966,3353216,1376,pulmonary|radiologic procedures / bronchoscopy...,True
1072967,3353251,12374,pulmonary|radiologic procedures / bronchoscopy...,True
1072968,3353251,186,pulmonary|radiologic procedures / bronchoscopy...,False
1072969,3353251,108,pulmonary|radiologic procedures / bronchoscopy...,False


### non invasive ventilation

In [21]:
type_3_ventilation_categories_1 = ["bi-pap", "ambubag"]
type_3_ventilation_1 = vent_settings_interest[vent_settings_interest["string"].isin(type_3_ventilation_categories_1)]
type_3_ventilation_1

,patientunitstayid,charttime,string,activeupondischarge
18195230,250972,805,bi-pap,None
22789420,404683,30,bi-pap,None
23464416,423397,32,bi-pap,None
25868948,490411,1042,bi-pap,None
25869437,490411,982,bi-pap,None
83979877,1743403,3257,ambubag,None
84054177,1750765,620,ambubag,None
84251966,1770776,8302,ambubag,None
84726168,1819019,305,ambubag,None
84758722,1822378,330,ambubag,None


In [22]:
type_3_ventilation_categories_2 = ["ipap", "niv", "epap", "mask leak", "volume assured", "non-invasive ventilation", "cpap"]
type_3_ventilation_2 = vent_settings_interest[vent_settings_interest["string"].str.contains(("|").join(type_3_ventilation_categories_2))]
type_3_ventilation_2

,patientunitstayid,charttime,string,activeupondischarge
11773905,208036,18,bipap/cpap,None
18080618,242757,52,bipap/cpap,None
18081207,242816,1719,cpap,None
18082472,243026,184,bipap/cpap,None
18084885,243334,1430,bipap/cpap,None
...,...,...,...,...
175998614,3352827,81,pulmonary|ventilation and oxygenation|non-inva...,False
175998622,3352827,1010,pulmonary|ventilation and oxygenation|non-inva...,False
175998731,3352964,2142,pulmonary|ventilation and oxygenation|non-inva...,True
175998732,3352964,206,pulmonary|ventilation and oxygenation|non-inva...,False


In [23]:
type_3_ventilation = pd.concat([type_3_ventilation_1, type_3_ventilation_2], ignore_index=True)
type_3_ventilation

,patientunitstayid,charttime,string,activeupondischarge
0,250972,805,bi-pap,None
1,404683,30,bi-pap,None
2,423397,32,bi-pap,None
3,490411,1042,bi-pap,None
4,490411,982,bi-pap,None
...,...,...,...,...
343639,3352827,81,pulmonary|ventilation and oxygenation|non-inva...,False
343640,3352827,1010,pulmonary|ventilation and oxygenation|non-inva...,False
343641,3352964,2142,pulmonary|ventilation and oxygenation|non-inva...,True
343642,3352964,206,pulmonary|ventilation and oxygenation|non-inva...,False


### either invasive or non-invasive ventilation

In [24]:
type_2_ventilation_categories_1 = [
    "flowtrigger", "peep", "tv/kg ibw", "mean airway pressure", "peak insp. pressure", "exhaled mv", "exhaled tv (machine)", "exhaled tv (patient)",
    "flow sensitivity", "peak flow", "f total", "pressure to trigger ps", "adult con setting set rr", "adult con setting set vt", "vti", "exhaled vt",
    "adult con alarms hi press alarm", "mve", "respiratory phase", "inspiratory pressure, set", "a1: high exhaled vt", "set fraction of inspired oxygen (fio2)",
    "insp flow (l/min)", "adult con setting spont exp vt", "spont tv", "pulse ox results vt", "vt spontaneous (ml)", "peak pressure", "ltv1200", "tc"
]
type_2_ventilation_1 = vent_settings_interest[vent_settings_interest["string"].isin(type_2_ventilation_categories_1)]
type_2_ventilation_1

,patientunitstayid,charttime,string,activeupondischarge
27140365,523739,3166,tc,None
46966665,1001599,1584,tc,None
46966674,1001599,1464,tc,None
46967630,1001599,1704,tc,None
48156239,1030115,171,tc,None
...,...,...,...,...
171772398,3335097,15149,vti,None
171772399,3335097,15149,exhaled tv (machine),None
171772403,3335097,15149,peak insp. pressure,None
171772404,3335097,15149,exhaled mv,None


In [25]:
mask = vent_settings_interest["string"].str.contains("vent") & ~vent_settings_interest["string"].str.contains("hyperventilat")
type_2_ventilation_2 = vent_settings_interest[mask]
type_2_ventilation_2

,patientunitstayid,charttime,string,activeupondischarge
119261,142723,1137,ventilator,None
190627,143325,605,ventilator,None
190703,143325,1685,ventilator,None
379004,144297,17681,ventilator,None
379693,144297,10421,ventilator,None
...,...,...,...,...
175999293,3353251,4080,pulmonary|ventilation and oxygenation|mechanic...,False
175999295,3353251,1201,pulmonary|ventilation and oxygenation|mechanic...,False
175999304,3353251,108,pulmonary|ventilation and oxygenation|mechanic...,False
175999309,3353251,2653,pulmonary|ventilation and oxygenation|mechanic...,False


In [26]:
type_2_ventilation_categories_3 = [
     "tidal", "flow rate", "minute volume", "leak", "pressure support", "peep", "tidal volume"
]
type_2_ventilation_3 = vent_settings_interest[vent_settings_interest["string"].str.contains(("|").join(type_2_ventilation_categories_3))]
type_2_ventilation_3

,patientunitstayid,charttime,string,activeupondischarge
23518053,425741,8748,40% ac 16 380 peep 8,None
23703105,430890,5585,ventilator/ac20/peep 10,None
23703286,430890,5465,ventilator/ac20/peep 10,None
23703548,430890,5942,ventilator/ac20/peep 10,None
23704141,430890,5765,ventilator/ac20/peep 10,None
...,...,...,...,...
175998774,3352986,51720,pulmonary|ventilation and oxygenation|mechanic...,True
175998829,3352986,43024,pulmonary|ventilation and oxygenation|mechanic...,False
175998835,3352986,41600,pulmonary|ventilation and oxygenation|mechanic...,False
175998841,3352986,34356,pulmonary|ventilation and oxygenation|mechanic...,False


In [27]:
type_2_ventilation = pd.concat([type_2_ventilation_1, type_2_ventilation_2, type_2_ventilation_3], ignore_index=True)
type_2_ventilation

,patientunitstayid,charttime,string,activeupondischarge
0,523739,3166,tc,None
1,1001599,1584,tc,None
2,1001599,1464,tc,None
3,1001599,1704,tc,None
4,1030115,171,tc,None
...,...,...,...,...
11953574,3352986,51720,pulmonary|ventilation and oxygenation|mechanic...,True
11953575,3352986,43024,pulmonary|ventilation and oxygenation|mechanic...,False
11953576,3352986,41600,pulmonary|ventilation and oxygenation|mechanic...,False
11953577,3352986,34356,pulmonary|ventilation and oxygenation|mechanic...,False


### supplemental oxygen

In [28]:
type_1_ventilation_categories_1 = [
    "t-piece", "blow-by", "oxyhood", "nc", "oxymizer", "hfnc", "oximizer", "high flow", "oxymask", "nch", "hi flow", "hiflow", 
    "hhfnc", "nasal canula", "face tent", "high flow mask", "aerosol mask", "venturi mask", "cool aerosol mask", "simple mask", "face mask"
]
type_1_ventilation_1 = vent_settings_interest[vent_settings_interest["string"].isin(type_1_ventilation_categories_1)]
type_1_ventilation_1

,patientunitstayid,charttime,string,activeupondischarge
12463326,211698,1300,nc,None
14515261,222376,10571,venturi mask,None
14998561,224911,26,cool aerosol mask,None
18074285,242290,780,nc,None
18075008,242429,58,nc,None
...,...,...,...,...
147300841,3244086,27,nc,None
147315661,3244237,1401,nc,None
147355368,3244829,-3,nc,None
147571156,3247055,2108,nc,None


In [29]:
type_1_ventilation_categories_2 = ["nasal cannula", "non-rebreather", "nasal mask", "face tent"]
type_1_ventilation_2 = vent_settings_interest[vent_settings_interest["string"].str.contains(("|").join(type_1_ventilation_categories_2))]
type_1_ventilation_2

,patientunitstayid,charttime,string,activeupondischarge
2329833,155221,0,nasal cannula,None
3431681,160709,1,nasal cannula,None
4196435,164866,-201,nasal cannula,None
4659955,167227,8335,nasal cannula,None
4675143,167346,1519,nasal cannula,None
...,...,...,...,...
175998933,3353094,5572,pulmonary|ventilation and oxygenation|oxygen t...,True
175998937,3353094,2556,pulmonary|ventilation and oxygenation|oxygen t...,False
175998938,3353094,3597,pulmonary|ventilation and oxygenation|oxygen t...,False
175998958,3353094,412,pulmonary|ventilation and oxygenation|oxygen t...,False


In [30]:
type_1_ventilation = pd.concat([type_1_ventilation_1, type_1_ventilation_2], ignore_index=True)
type_1_ventilation

,patientunitstayid,charttime,string,activeupondischarge
0,211698,1300,nc,None
1,222376,10571,venturi mask,None
2,224911,26,cool aerosol mask,None
3,242290,780,nc,None
4,242429,58,nc,None
...,...,...,...,...
584787,3353094,5572,pulmonary|ventilation and oxygenation|oxygen t...,True
584788,3353094,2556,pulmonary|ventilation and oxygenation|oxygen t...,False
584789,3353094,3597,pulmonary|ventilation and oxygenation|oxygen t...,False
584790,3353094,412,pulmonary|ventilation and oxygenation|oxygen t...,False


### oxygen therapy but unknown what type

In [31]:
type_0_ventilation_categories_1 = ["pressure support", "rr spont", "ps", "insp cycle off (%)", "trach mask/collar"]
type_0_ventilation_1 = vent_settings_interest[vent_settings_interest["string"].isin(type_0_ventilation_categories_1)]
type_0_ventilation_1

,patientunitstayid,charttime,string,activeupondischarge
25631568,484604,23413,pressure support,None
25631777,484604,23473,pressure support,None
25632351,484604,23713,pressure support,None
25633215,484604,23403,pressure support,None
25633835,484604,23653,pressure support,None
...,...,...,...,...
171772343,3331456,4948,pressure support,None
171772352,3348331,57927,pressure support,None
171772358,3346355,11946,ps,None
171772366,3350620,35660,pressure support,None


In [32]:
type_0_ventilation_categories_2 = ["spontaneous", "oxygen therapy"]
type_0_ventilation_2 = vent_settings_interest[vent_settings_interest["string"].str.contains(("|").join(type_0_ventilation_categories_2))]
type_0_ventilation_2

,patientunitstayid,charttime,string,activeupondischarge
27074294,522114,10771,"pressure support 10, spontaneous",None
27074380,522114,10681,"pressure support 14, spontaneous",None
27075811,522114,10801,"pressure support 10, spontaneous",None
27077065,522114,10741,"pressure support 14, spontaneous",None
106831905,2669251,3749,4-->(e4) spontaneous,None
...,...,...,...,...
175998958,3353094,412,pulmonary|ventilation and oxygenation|oxygen t...,False
175998965,3353094,417,pulmonary|ventilation and oxygenation|oxygen t...,False
175999089,3353127,996,pulmonary|ventilation and oxygenation|oxygen t...,True
175999090,3353127,124,pulmonary|ventilation and oxygenation|oxygen t...,False


In [33]:
type_0_ventilation = pd.concat([type_0_ventilation_1, type_0_ventilation_2], ignore_index=True)
type_0_ventilation

,patientunitstayid,charttime,string,activeupondischarge
0,484604,23413,pressure support,None
1,484604,23473,pressure support,None
2,484604,23713,pressure support,None
3,484604,23403,pressure support,None
4,484604,23653,pressure support,None
...,...,...,...,...
740233,3353094,412,pulmonary|ventilation and oxygenation|oxygen t...,False
740234,3353094,417,pulmonary|ventilation and oxygenation|oxygen t...,False
740235,3353127,996,pulmonary|ventilation and oxygenation|oxygen t...,True
740236,3353127,124,pulmonary|ventilation and oxygenation|oxygen t...,False


### supplemental oxygen therapy (i.e. more oxygen than in room air is administered)

In [34]:
type_negative_1_ventilation = vent_settings_interest[vent_settings_interest["string"].isin(["lpm o2"])]
type_negative_1_ventilation

,patientunitstayid,charttime,string,activeupondischarge
151604243,231507,-3297,lpm o2,None
151604257,196110,201,lpm o2,None
151604259,184757,2922,lpm o2,None
151604269,211096,7778,lpm o2,None
151604275,152899,5050,lpm o2,None
...,...,...,...,...
170272001,3228278,9120,lpm o2,None
170272044,3237428,4030,lpm o2,None
170272144,3224033,-10527,lpm o2,None
170272162,3206804,1386,lpm o2,None


### assign oxygen_therapy_type

In [35]:
type_4_ventilation["oxygen_therapy_type"] = 4
type_3_ventilation["oxygen_therapy_type"] = 3
type_2_ventilation["oxygen_therapy_type"] = 2
type_1_ventilation["oxygen_therapy_type"] = 1
type_0_ventilation["oxygen_therapy_type"] = 0
type_negative_1_ventilation["oxygen_therapy_type"] = -1

In [36]:
ventilation_oxygen_types  = pd.concat([
    type_4_ventilation, type_3_ventilation, type_2_ventilation, type_1_ventilation, type_0_ventilation, type_negative_1_ventilation], ignore_index=True)
ventilation_oxygen_types = ventilation_oxygen_types[ventilation_oxygen_types["charttime"]>=-60]
ventilation_oxygen_types

,patientunitstayid,charttime,string,activeupondischarge,oxygen_therapy_type
0,221468,4679,plateau pressure,None,4
1,181209,34983,plateau pressure,None,4
2,189335,13086,plateau pressure,None,4
3,189335,13086,pressure control,None,4
4,172049,916,plateau pressure,None,4
...,...,...,...,...,...
15714050,3216895,4849,lpm o2,None,-1
15714051,3228278,9120,lpm o2,None,-1
15714052,3237428,4030,lpm o2,None,-1
15714054,3206804,1386,lpm o2,None,-1


In [37]:
vent_settings_interest = vent_settings_interest[vent_settings_interest["charttime"]>=-60]
ventsettings0 = vent_settings_interest.merge(ventilation_oxygen_types, on=["patientunitstayid", "charttime", "string", "activeupondischarge"],  how="left")
ventsettings0

,patientunitstayid,charttime,string,activeupondischarge,oxygen_therapy_type
0,141924,5714,36.6,None,NaN
1,141924,12964,x,None,NaN
2,141924,3844,97,None,NaN
3,141924,124,106,None,NaN
4,141924,1489,71,None,NaN
...,...,...,...,...,...
170924694,3353251,291,pulmonary|radiologic procedures / bronchoscopy...,False,4.0
170924695,3353251,291,pulmonary|ventilation and oxygenation|mechanic...,False,2.0
170924696,3353254,41,cardiovascular|intravenous fluid|normal saline...,True,NaN
170924697,3353254,41,cardiovascular|intravenous fluid|blood product...,True,NaN


In [38]:
ventsettings0 = ventsettings0[["patientunitstayid", "charttime", "activeupondischarge", "oxygen_therapy_type"]]

## Ventilation settings 1 from nurse with unclear type

In [39]:
mask = nurse_charting["nursingchartvalue"].apply(is_numeric)
nurse_charting_numeric = nurse_charting[mask]
nurse_charting_numeric = nurse_charting_numeric[nurse_charting_numeric["nursingchartoffset"]>=-60]
nurse_charting_numeric["nursingchartvalue"] = nurse_charting_numeric["nursingchartvalue"].astype(float)
nurse_charting_numeric = nurse_charting_numeric[(nurse_charting_numeric["nursingchartvalue"]>0) & (nurse_charting_numeric["nursingchartvalue"]<=100)]
nurse_charting_numeric = nurse_charting_numeric[nurse_charting_numeric["nursingchartcelltypevallabel"].isin(["O2 L/%"])]
nurse_charting_numeric

,nursingchartid,patientunitstayid,nursingchartoffset,nursingchartentryoffset,nursingchartcelltypecat,nursingchartcelltypevallabel,nursingchartcelltypevalname,nursingchartvalue
118517,143534115,142723,1137,1124,Vital Signs,O2 L/%,O2 L/%,60.0
190569,197697876,143325,605,632,Vital Signs,O2 L/%,O2 L/%,30.0
190667,179642537,143325,1685,1713,Vital Signs,O2 L/%,O2 L/%,60.0
378283,287965059,144297,17681,17658,Vital Signs,O2 L/%,O2 L/%,60.0
389618,89369059,144297,59381,59401,Vital Signs,O2 L/%,O2 L/%,40.0
...,...,...,...,...,...,...,...,...
151470510,2300968862,3353263,410,410,Vital Signs,O2 L/%,O2 L/%,2.0
151470512,2304778352,3353263,470,470,Vital Signs,O2 L/%,O2 L/%,2.0
151470514,2297229836,3353263,5,5,Vital Signs,O2 L/%,O2 L/%,2.0
151470540,2299382038,3353263,770,770,Vital Signs,O2 L/%,O2 L/%,2.0


In [40]:
ventsettings1 = nurse_charting_numeric[["patientunitstayid", "nursingchartoffset"]]
ventsettings1 = ventsettings1.rename(columns={"nursingchartoffset": "charttime"})
ventsettings1["activeupondischarge"] = None
ventsettings1["oxygen_therapy_type"] = -1
ventsettings1

,patientunitstayid,charttime,activeupondischarge,oxygen_therapy_type
118517,142723,1137,None,-1
190569,143325,605,None,-1
190667,143325,1685,None,-1
378283,144297,17681,None,-1
389618,144297,59381,None,-1
...,...,...,...,...
151470510,3353263,410,None,-1
151470512,3353263,470,None,-1
151470514,3353263,5,None,-1
151470540,3353263,770,None,-1


## Ventilation settings 2 from resp fiO2

In [41]:
mask = resp_chart["respchartvalue"].apply(is_numeric)
resp_chart_numeric = resp_chart[mask]
resp_chart_numeric = resp_chart_numeric[resp_chart_numeric["respchartoffset"]>=-60]
resp_chart_numeric["respchartvalue"] = resp_chart_numeric["respchartvalue"].astype(float)
resp_chart_numeric = resp_chart_numeric[resp_chart_numeric["respchartvaluelabel"].isin(["fio2", "fio2 (%)"])]

In [42]:
mask = (resp_chart_numeric["respchartvalue"]<0.2) | ((resp_chart_numeric["respchartvalue"]>0.22) & (resp_chart_numeric["respchartvalue"]<20)) | (resp_chart_numeric["respchartvalue"]>22)
resp_chart_numeric = resp_chart_numeric[mask]
resp_chart_numeric

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue
1,4,221468,4679,4679,respFlowSettings,fio2,40.0
15,60,152518,27557,27557,respFlowSettings,fio2,50.0
22,81,204319,1917,1917,respFlowSettings,fio2,60.0
24,85,241257,4468,4468,respFlowSettings,fio2,70.0
26,105,192236,27137,27137,respFlowSettings,fio2,40.0
...,...,...,...,...,...,...,...
20168144,255567477,3352737,1906,1906,respFlowSettings,fio2,90.0
20168150,255567494,3349895,5786,5786,respFlowSettings,fio2,30.0
20168170,255567517,3335097,15149,15149,respFlowSettings,fio2,40.0
20168174,255567527,3331083,126,126,respFlowSettings,fio2,40.0


In [43]:
resp_chart_numeric["oxygen_therapy_type"] = 0
resp_chart_numeric.loc[(resp_chart_numeric["respchartvalue"]>0.22)&(resp_chart_numeric["respchartvalue"]<=1), "resp_chart_numeric"] = -1
resp_chart_numeric.loc[resp_chart_numeric["respchartvalue"]>22, "resp_chart_numeric"] = -1
resp_chart_numeric

,respchartid,patientunitstayid,respchartoffset,respchartentryoffset,respcharttypecat,respchartvaluelabel,respchartvalue,oxygen_therapy_type,resp_chart_numeric
1,4,221468,4679,4679,respFlowSettings,fio2,40.0,0,-1.0
15,60,152518,27557,27557,respFlowSettings,fio2,50.0,0,-1.0
22,81,204319,1917,1917,respFlowSettings,fio2,60.0,0,-1.0
24,85,241257,4468,4468,respFlowSettings,fio2,70.0,0,-1.0
26,105,192236,27137,27137,respFlowSettings,fio2,40.0,0,-1.0
...,...,...,...,...,...,...,...,...,...
20168144,255567477,3352737,1906,1906,respFlowSettings,fio2,90.0,0,-1.0
20168150,255567494,3349895,5786,5786,respFlowSettings,fio2,30.0,0,-1.0
20168170,255567517,3335097,15149,15149,respFlowSettings,fio2,40.0,0,-1.0
20168174,255567527,3331083,126,126,respFlowSettings,fio2,40.0,0,-1.0


In [44]:
resp_chart_numeric["activeupondischarge"] = None

In [45]:
ventsettings2 = resp_chart_numeric[["patientunitstayid", "respchartoffset", "activeupondischarge", "oxygen_therapy_type"]]
ventsettings2 = ventsettings1.rename(columns={"nursingchartoffset": "charttime"})
ventsettings2

,patientunitstayid,charttime,activeupondischarge,oxygen_therapy_type
118517,142723,1137,None,-1
190569,143325,605,None,-1
190667,143325,1685,None,-1
378283,144297,17681,None,-1
389618,144297,59381,None,-1
...,...,...,...,...
151470510,3353263,410,None,-1
151470512,3353263,470,None,-1
151470514,3353263,5,None,-1
151470540,3353263,770,None,-1


## Merge ventilation settings 0, 1, 2

In [46]:
ventsettings = pd.concat([ventsettings0, ventsettings1, ventsettings2], ignore_index=True)
ventsettings

,patientunitstayid,charttime,activeupondischarge,oxygen_therapy_type
0,141924,5714,None,NaN
1,141924,12964,None,NaN
2,141924,3844,None,NaN
3,141924,124,None,NaN
4,141924,1489,None,NaN
...,...,...,...,...
174624068,3353263,410,None,-1.0
174624069,3353263,470,None,-1.0
174624070,3353263,5,None,-1.0
174624071,3353263,770,None,-1.0


In [47]:
ventsettings = ventsettings[~ventsettings["oxygen_therapy_type"].isna()]

In [48]:
ventsettings.loc[ventsettings["activeupondischarge"].isna(), "activeupondischarge"] = False

In [49]:
ventsettings["supp_oxygen"] = 0
ventsettings.loc[ventsettings["oxygen_therapy_type"]==-1, "supp_oxygen"] = 1
ventsettings

,patientunitstayid,charttime,activeupondischarge,oxygen_therapy_type,supp_oxygen
109623,142723,1137,False,2.0,0
175583,143325,605,False,2.0,0
175616,143325,1685,False,2.0,0
350038,144297,17681,False,2.0,0
350727,144297,10421,False,2.0,0
...,...,...,...,...,...
174624068,3353263,410,False,-1.0,1
174624069,3353263,470,False,-1.0,1
174624070,3353263,5,False,-1.0,1
174624071,3353263,770,False,-1.0,1


In [50]:
# ensure charttime is unique
ventsettings_grouped = ventsettings.groupby(["patientunitstayid", "charttime"]).agg({"oxygen_therapy_type": "max", "activeupondischarge": "max", "supp_oxygen": "max"})
ventsettings_grouped

oxygen_therapy_type  activeupondischarge  \
patientunitstayid charttime                                             
141168            396                       -1.0                False   
                  506                        1.0                False   
                  563                       -1.0                False   
                  606                       -1.0                False   
                  666                       -1.0                False   
...                                          ...                  ...   
3353263           530                       -1.0                False   
                  590                       -1.0                False   
                  650                       -1.0                False   
                  710                       -1.0                False   
                  770                       -1.0                False   

                             supp_oxygen  
patientunitstayid charttime               
141168            396                  1  
                  506                  1  
                  563                  1  
                  606                  1  
                  666                  1  
...                                  ...  
3353263           530                  1  
                  590                  1  
                  650                  1  
                  710                  1  
                  770                  1  

[5622224 rows x 3 columns]

### vd0

In [51]:
vd0 = ventsettings_grouped.reset_index(level=["charttime"])
vd0

,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen
patientunitstayid,,,,
141168,396,-1.0,False,1
141168,506,1.0,False,1
141168,563,-1.0,False,1
141168,606,-1.0,False,1
141168,666,-1.0,False,1
...,...,...,...,...
3353263,530,-1.0,False,1
3353263,590,-1.0,False,1
3353263,650,-1.0,False,1


In [52]:
vd0["charttime_lag"] = vd0.groupby(level=["patientunitstayid"])["charttime"].shift(1)
vd0

,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen,charttime_lag
patientunitstayid,,,,,
141168,396,-1.0,False,1,NaN
141168,506,1.0,False,1,396.0
141168,563,-1.0,False,1,506.0
141168,606,-1.0,False,1,563.0
141168,666,-1.0,False,1,606.0
...,...,...,...,...,...
3353263,530,-1.0,False,1,470.0
3353263,590,-1.0,False,1,530.0
3353263,650,-1.0,False,1,590.0


### vd1

In [53]:
vd1 = vd0.copy(True)

In [54]:
vd1["new_event"] = 0
vd1.loc[vd1["charttime_lag"].isna(), "new_event"] = 1
vd1["charttime_lag"] = vd1["charttime_lag"].fillna(0)
vd1.loc[(vd1["charttime"]-vd1["charttime_lag"])>24*60, "new_event"] = 1
vd1

,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen,charttime_lag,new_event
patientunitstayid,,,,,,
141168,396,-1.0,False,1,0.0,1
141168,506,1.0,False,1,396.0,0
141168,563,-1.0,False,1,506.0,0
141168,606,-1.0,False,1,563.0,0
141168,666,-1.0,False,1,606.0,0
...,...,...,...,...,...,...
3353263,530,-1.0,False,1,470.0,0
3353263,590,-1.0,False,1,530.0,0
3353263,650,-1.0,False,1,590.0,0


In [55]:
vd1 = vd1[["charttime", "oxygen_therapy_type", "activeupondischarge", "supp_oxygen", "new_event"]]
vd1 = vd1.reset_index()
vd1

,patientunitstayid,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen,new_event
0,141168,396,-1.0,False,1,1
1,141168,506,1.0,False,1,0
2,141168,563,-1.0,False,1,0
3,141168,606,-1.0,False,1,0
4,141168,666,-1.0,False,1,0
...,...,...,...,...,...,...
5622219,3353263,530,-1.0,False,1,0
5622220,3353263,590,-1.0,False,1,0
5622221,3353263,650,-1.0,False,1,0
5622222,3353263,710,-1.0,False,1,0


### vd2

In [56]:
ventnum = vd1.groupby("patientunitstayid")["new_event"].cumsum()
ventnum

0          1
1          1
2          1
3          1
4          1
          ..
5622219    1
5622220    1
5622221    1
5622222    1
5622223    1
Name: new_event, Length: 5622224, dtype: int64

In [57]:
vd2 = vd1.copy(True)
vd2["ventnum"] = ventnum
vd2

,patientunitstayid,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen,new_event,ventnum
0,141168,396,-1.0,False,1,1,1
1,141168,506,1.0,False,1,0,1
2,141168,563,-1.0,False,1,0,1
3,141168,606,-1.0,False,1,0,1
4,141168,666,-1.0,False,1,0,1
...,...,...,...,...,...,...,...
5622219,3353263,530,-1.0,False,1,0,1
5622220,3353263,590,-1.0,False,1,0,1
5622221,3353263,650,-1.0,False,1,0,1
5622222,3353263,710,-1.0,False,1,0,1


### vd3

In [58]:
vd3 = vd2.merge(icu_stays, on="patientunitstayid", how="left")
vd3

,patientunitstayid,charttime,oxygen_therapy_type,activeupondischarge,supp_oxygen,new_event,ventnum,patienthealthsystemstayid,gender,age,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141168,396,-1.0,False,1,1,1,128919,Female,70,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
1,141168,506,1.0,False,1,0,1,128919,Female,70,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
2,141168,563,-1.0,False,1,0,1,128919,Female,70,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
3,141168,606,-1.0,False,1,0,1,128919,Female,70,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
4,141168,666,-1.0,False,1,0,1,128919,Female,70,...,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622219,3353263,530,-1.0,False,1,0,1,2743110,Male,35,...,Emergency Department,1,admit,100.8,100.8,20:38:00,928,Telemetry,Alive,035-22638
5622220,3353263,590,-1.0,False,1,0,1,2743110,Male,35,...,Emergency Department,1,admit,100.8,100.8,20:38:00,928,Telemetry,Alive,035-22638
5622221,3353263,650,-1.0,False,1,0,1,2743110,Male,35,...,Emergency Department,1,admit,100.8,100.8,20:38:00,928,Telemetry,Alive,035-22638
5622222,3353263,710,-1.0,False,1,0,1,2743110,Male,35,...,Emergency Department,1,admit,100.8,100.8,20:38:00,928,Telemetry,Alive,035-22638


In [59]:
vd3["vent_start"] = vd3["charttime"]

In [60]:
vent_start_end = vd3.groupby(["patientunitstayid", "ventnum"]).agg({
    "activeupondischarge": "max", "vent_start": "min", "charttime": "max", "unitdischargeoffset": "max",
    "oxygen_therapy_type": "max", "supp_oxygen": "max"
})
vent_start_end

,,activeupondischarge,vent_start,charttime,unitdischargeoffset,oxygen_therapy_type,supp_oxygen
patientunitstayid,ventnum,,,,,,
141168,1,False,396,2329,3596,4.0,1
141179,1,False,12,642,2042,-1.0,1
141194,1,False,-3,6127,4813,-1.0,1
141196,1,False,840,5303,1463,-1.0,1
141227,1,False,-13,1573,1652,4.0,1
...,...,...,...,...,...,...,...
3353235,1,True,16,331,1069,2.0,1
3353237,1,True,-14,1201,1269,2.0,1
3353251,1,True,0,16183,16259,4.0,1


In [61]:
vent_start_end["vent_end"] = vent_start_end["charttime"] + 60

In [62]:
mask = vent_start_end["activeupondischarge"] & (vent_start_end["charttime"]+60<vent_start_end["unitdischargeoffset"])
vent_start_end.loc[mask, "vent_end"] = vent_start_end.loc[mask, "unitdischargeoffset"]

In [63]:
vent_start_end

,,activeupondischarge,vent_start,charttime,unitdischargeoffset,oxygen_therapy_type,supp_oxygen,vent_end
patientunitstayid,ventnum,,,,,,,
141168,1,False,396,2329,3596,4.0,1,2389
141179,1,False,12,642,2042,-1.0,1,702
141194,1,False,-3,6127,4813,-1.0,1,6187
141196,1,False,840,5303,1463,-1.0,1,5363
141227,1,False,-13,1573,1652,4.0,1,1633
...,...,...,...,...,...,...,...,...
3353235,1,True,16,331,1069,2.0,1,1069
3353237,1,True,-14,1201,1269,2.0,1,1269
3353251,1,True,0,16183,16259,4.0,1,16259


In [64]:
vent_start_end["oxygen_therapy_type"].value_counts()

-1.0    59363
 4.0    48484
 2.0    45392
 3.0    16862
 1.0     8985
 0.0     5943
Name: oxygen_therapy_type, dtype: int64

In [65]:
vent_start_end.to_csv("/N/project/waveform_mortality/xiang/Projects/hr_sbp_contour/data/mv_raw.csv")